In [13]:
from typing import Annotated
from typing_extensions import TypedDict
from dotenv import load_dotenv

load_dotenv()

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from langchain.tools import tool


Tool definition

In [14]:
@tool
def km_to_meters(km: float) -> float:
    """Convert kilometers to meters"""
    return km * 1000


LLM (Groq) + bind tool

In [15]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

llm_with_tools = llm.bind_tools([km_to_meters])


State

In [16]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


LangGraph node (THIS IS TOOL CALLING)

In [25]:
def chatbot(state: State) -> State:
    response = llm_with_tools.invoke(state["messages"])

    # If LLM decided to call a tool
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        result = km_to_meters.invoke(tool_call["args"])

        return {
            "messages": [
                response,
                {
                    "role": "tool",
                    "content": str(result)
                }
            ]
        }

    # Normal text response
    return {"messages": [response]}


Graph definition

In [26]:
builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()


Interactive loop

In [27]:
state = None

while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break

    if state is None:
        state = {"messages": [{"role": "user", "content": user_input}]}
    else:
        state["messages"].append({"role": "user", "content": user_input})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)


Bot: I think you meant to say "add two numbers". Here's a simple Python function to do that:

```python
def add_numbers(a, b):
    return a + b

# Example usage:
num1 = 5
num2 = 7
result = add_numbers(num1, num2)
print(f"The sum of {num1} and {num2} is: {result}")
```

This code defines a function `add_numbers` that takes two arguments `a` and `b`, adds them together, and returns the result. The example usage shows how to call this function with two numbers, `num1` and `num2`, and print the result.
